In [1]:
import numpy as np

In [2]:
#mdm_data = np.load('/home/rishabhswapnil/priorMDM/save/my_humanml_trans_enc_512/DoubleTake_samples_my_humanml_trans_enc_512_000200000_seed10_dt_text_example_handshake_20_double_take_blend_10_skipSteps_100/results.npy', allow_pickle=True)


In [3]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# https://pytorch3d.readthedocs.io/en/latest/_modules/pytorch3d/transforms/rotation_conversions.html#matrix_to_quaternion

import torch
import torch.nn.functional as F

def _copysign(a, b):
    """
    Return a tensor where each element has the absolute value taken from the,
    corresponding element of a, with sign taken from the corresponding
    element of b. This is like the standard copysign floating-point operation,
    but is not careful about negative 0 and NaN.
    Args:
        a: source tensor.
        b: tensor whose signs will be used, of the same shape as a.
    Returns:
        Tensor of the same shape as a with the signs of b.
    """
    signs_differ = (a < 0) != (b < 0)
    return torch.where(signs_differ, -a, a)

def _sqrt_positive_part(x):
    """
    Returns torch.sqrt(torch.max(0, x))
    but with a zero subgradient where x is 0.
    """
    ret = torch.zeros_like(x)
    positive_mask = x > 0
    ret[positive_mask] = torch.sqrt(x[positive_mask])
    return ret

def matrix_to_quaternion(matrix):
    """
    Convert rotations given as rotation matrices to quaternions.
    Args:
        matrix: Rotation matrices as tensor of shape (..., 3, 3).
    Returns:
        quaternions with real part first, as tensor of shape (..., 4).
    """
    if matrix.size(-1) != 3 or matrix.size(-2) != 3:
        raise ValueError(f"Invalid rotation matrix  shape f{matrix.shape}.")
    m00 = matrix[..., 0, 0]
    m11 = matrix[..., 1, 1]
    m22 = matrix[..., 2, 2]
    o0 = 0.5 * _sqrt_positive_part(1 + m00 + m11 + m22)
    x = 0.5 * _sqrt_positive_part(1 + m00 - m11 - m22)
    y = 0.5 * _sqrt_positive_part(1 - m00 + m11 - m22)
    z = 0.5 * _sqrt_positive_part(1 - m00 - m11 + m22)
    o1 = _copysign(x, matrix[..., 2, 1] - matrix[..., 1, 2])
    o2 = _copysign(y, matrix[..., 0, 2] - matrix[..., 2, 0])
    o3 = _copysign(z, matrix[..., 1, 0] - matrix[..., 0, 1])

    return torch.stack((o0, o1, o2, o3), -1)

def quaternion_to_axis_angle(quaternions):
    """
    Convert rotations given as quaternions to axis/angle.
    Args:
        quaternions: quaternions with real part first,
            as tensor of shape (..., 4).
    Returns:
        Rotations given as a vector in axis angle form, as a tensor
            of shape (..., 3), where the magnitude is the angle
            turned anticlockwise in radians around the vector's
            direction.
    """
    norms = torch.norm(quaternions[..., 1:], p=2, dim=-1, keepdim=True)
    half_angles = torch.atan2(norms, quaternions[..., :1])
    angles = 2 * half_angles
    eps = 1e-6
    small_angles = angles.abs() < eps
    sin_half_angles_over_angles = torch.empty_like(angles)
    sin_half_angles_over_angles[~small_angles] = (
        torch.sin(half_angles[~small_angles]) / angles[~small_angles]
    )
    # for x small, sin(x/2) is about x/2 - (x/2)^3/6
    # so sin(x/2)/x is about 1/2 - (x*x)/48
    sin_half_angles_over_angles[small_angles] = (
        0.5 - (angles[small_angles] * angles[small_angles]) / 48
    )

    return quaternions[..., 1:] / sin_half_angles_over_angles


def rotation_6d_to_matrix(d6):
    """ 
    Converts 6D rotation representation by Zhou et al. [1] to rotation matrix 
    using Gram--Schmidt orthogonalisation per Section B of [1]. 
    Args: 
        d6: 6D rotation representation, of size (*, 6) 

    Returns: 
        batch of rotation matrices of size (*, 3, 3) 

    [1] Zhou, Y., Barnes, C., Lu, J., Yang, J., & Li, H. 
    On the Continuity of Rotation Representations in Neural Networks. 
    IEEE Conference on Computer Vision and Pattern Recognition, 2019. 
    Retrieved from http://arxiv.org/abs/1812.07035 
    """ 

    a1, a2 = d6[..., :3], d6[..., 3:] 
    b1 = F.normalize(a1, dim=-1) 
    b2 = a2 - (b1 * a2).sum(-1, keepdim=True) * b1 
    b2 = F.normalize(b2, dim=-1) 
    b3 = torch.cross(b1, b2, dim=-1)

    return torch.stack((b1, b2, b3), dim=-2) 

def matrix_to_axis_angle(matrix):
    """ 
    Convert rotations given as rotation matrices to axis/angle. 

    Args: 
        matrix: Rotation matrices as tensor of shape (..., 3, 3). 

    Returns: 
        Rotations given as a vector in axis angle form, as a tensor 
            of shape (..., 3), where the magnitude is the angle 
            turned anticlockwise in radians around the vector's 
            direction. 
    """

    return quaternion_to_axis_angle(matrix_to_quaternion(matrix)) 

def matrix_to_rotation_6d(matrix: torch.Tensor) -> torch.Tensor:
    """
    Converts rotation matrices to 6D rotation representation by Zhou et al. [1]
    by dropping the last row. Note that 6D representation is not unique.
    Args:
        matrix: batch of rotation matrices of size (*, 3, 3)

    Returns:
        6D rotation representation, of size (*, 6)

    [1] Zhou, Y., Barnes, C., Lu, J., Yang, J., & Li, H.
    On the Continuity of Rotation Representations in Neural Networks.
    IEEE Conference on Computer Vision and Pattern Recognition, 2019.
    Retrieved from http://arxiv.org/abs/1812.07035
    """
    batch_dim = matrix.size()[:-2]
    return matrix[..., :2, :].clone().reshape(batch_dim + (6,))

def axis_angle_to_matrix(axis_angle: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as axis/angle to rotation matrices.

    Args:
        axis_angle: Rotations given as a vector in axis angle form,
            as a tensor of shape (..., 3), where the magnitude is
            the angle turned anticlockwise in radians around the
            vector's direction.

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """
    return quaternion_to_matrix(axis_angle_to_quaternion(axis_angle))


def axis_angle_to_quaternion(axis_angle: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as axis/angle to quaternions.

    Args:
        axis_angle: Rotations given as a vector in axis angle form,
            as a tensor of shape (..., 3), where the magnitude is
            the angle turned anticlockwise in radians around the
            vector's direction.

    Returns:
        quaternions with real part first, as tensor of shape (..., 4).
    """
    angles = torch.norm(axis_angle, p=2, dim=-1, keepdim=True)
    half_angles = angles * 0.5
    eps = 1e-6
    small_angles = angles.abs() < eps
    sin_half_angles_over_angles = torch.empty_like(angles)
    sin_half_angles_over_angles[~small_angles] = (
        torch.sin(half_angles[~small_angles]) / angles[~small_angles]
    )
    # for x small, sin(x/2) is about x/2 - (x/2)^3/6
    # so sin(x/2)/x is about 1/2 - (x*x)/48
    sin_half_angles_over_angles[small_angles] = (
        0.5 - (angles[small_angles] * angles[small_angles]) / 48
    )
    quaternions = torch.cat(
        [torch.cos(half_angles), axis_angle * sin_half_angles_over_angles],
        dim=-1,
    )
    return quaternions


def quaternion_to_matrix(quaternions: torch.Tensor) -> torch.Tensor:
    """
    Convert rotations given as quaternions to rotation matrices.

    Args:
        quaternions: quaternions with real part first,
            as tensor of shape (..., 4).

    Returns:
        Rotation matrices as tensor of shape (..., 3, 3).
    """
    r, i, j, k = torch.unbind(quaternions, -1)
    two_s = 2.0 / (quaternions * quaternions).sum(-1)

    o = torch.stack(
        (
            1 - two_s * (j * j + k * k),
            two_s * (i * j - k * r),
            two_s * (i * k + j * r),
            two_s * (i * j + k * r),
            1 - two_s * (i * i + k * k),
            two_s * (j * k - i * r),
            two_s * (i * k - j * r),
            two_s * (j * k + i * r),
            1 - two_s * (i * i + j * j),
        ),
        -1,
    )
    return o.reshape(quaternions.shape[:-1] + (3, 3))


def _sqrt_positive_part(x: torch.Tensor) -> torch.Tensor:
    """
    Returns torch.sqrt(torch.max(0, x))
    but with a zero subgradient where x is 0.
    """
    ret = torch.zeros_like(x)
    positive_mask = x > 0
    ret[positive_mask] = torch.sqrt(x[positive_mask])
    return ret


In [4]:
pose_path = '/home/rishabhswapnil/priorMDM/save/my_humanml_trans_enc_512/DoubleTake_samples_my_humanml_trans_enc_512_000200000_seed10_dt_csv_example_handshake_20_double_take_blend_10_skipSteps_100/sample00_rep00_smpl_params.npy'
poses = np.load(pose_path, allow_pickle=True).item()['thetas']
transl = np.load(pose_path, allow_pickle=True).item()['root_translation']

In [5]:
pose_output = []

# Iterate over the third dimension of the poses array
for idx in range(poses.shape[2]):
    # Convert the slice to a PyTorch tensor and compute the rotation matrix
    rotation_matrix = rotation_6d_to_matrix(torch.tensor(poses[:, :, idx]))
    
    # Convert the rotation matrix to axis-angle form and convert to a NumPy array
    axis_angle = np.array(matrix_to_axis_angle(rotation_matrix))
    
    # Reshape the output and append to the list
    pose_output.append(axis_angle.reshape(-1))

# Convert the list to a NumPy array
pose_output_array = np.array(pose_output)

In [6]:
final_dict = {}
final_dict["smpl_trans"] = transl.reshape(-1,3)
final_dict["smpl_poses"] = pose_output_array


In [7]:
import pickle
file_path = '/home/rishabhswapnil/arah-release/aist_motion/motions/data_test.pkl'

# Open a file for writing. This will overwrite any existing file with the same name.
with open(file_path, 'wb') as file:
    # Use pickle to dump the dictionary into the file
    pickle.dump(final_dict, file)

print(f'Dictionary has been successfully saved and any existing file was overwritten at {file_path}')

Dictionary has been successfully saved and any existing file was overwritten at /home/rishabhswapnil/arah-release/aist_motion/motions/data_test.pkl


In [8]:
mdm_data = np.load('/home/rishabhswapnil/arah-release/aist_motion/motions/data_test.pkl', allow_pickle=True)


In [9]:
mdm_data["smpl_trans"].shape

(1370, 3)

In [10]:
#final_dict["smpl_trans"].shape

In [11]:
#temp_poses = np.array(matrix_to_axis_angle(rotation_6d_to_matrix(torch.tensor(poses[:, :, 1])))).reshape(-1)


In [12]:
"""temp_poses[0] = 0.
temp_poses[1] = 0.
temp_poses[2] = 0.
final_poses =np.append(final_poses,temp_poses,axis=1)"""

'temp_poses[0] = 0.\ntemp_poses[1] = 0.\ntemp_poses[2] = 0.\nfinal_poses =np.append(final_poses,temp_poses,axis=1)'

In [13]:
#final_dict["smpl_trans"].shape

In [14]:
#poses = np.array(matrix_to_axis_angle(rotation_6d_to_matrix(torch.tensor(self.poses[:, :, idx])))).reshape(-1)

In [15]:
#thetas = mdm_data.item().get("thetas")  # Get the joint